In [1]:
import logging

logger = logging.getLogger()

# format
formatter = logging.Formatter('[%(asctime)s] [%(levelname)s] %(message)s')

# add stream handler
handler = logging.StreamHandler(sys.stdout)                             
handler.setFormatter(formatter)
logger.addHandler(handler)

# enable all logging level
logger.setLevel(logging.INFO)

Definition for the board.

In [2]:
from ece437.ok import OKFrontPanel

path = "C:\\ytliu2\\ECE437\\Lab7\\Lab7.runs\\impl_1\\lab7_top.bit"
fp = OKFrontPanel(firmware_path=path)

Start control DRV8833 over PMOD port 1.

In [3]:
from ece437.presets import I2C_1_ENDPOINTS
from ece437.i2c import I2CController
from ece437.sensor import LSM303Accelerometer, LSM303Magnetometer
from ece437.pmod import PMOD, PMODType5Endpoints, PMODDirection
import time
import numpy as np

PMOD_1_ENDPOINTS = PMODType5Endpoints(
    RESET = 0x00,
    CONTROL = 0x03,
    TRIGGER_IN = 0x41,
    TRIGGER_OUT = 0x60,

    RESET_MASK = 2,

    START_MASK = 0,
    DONE_MASK = 2
)

with fp:
    print(f"serial={fp.serial_number}")

    with I2CController(fp, I2C_1_ENDPOINTS) as i2c1:
        acc = LSM303Accelerometer(i2c1)
        acc.configure() # defaults

        mag = LSM303Magnetometer(i2c1)
        mag.configure() # defaults
        
        acc_history = []
        mag_history = []

        with PMOD(fp, PMOD_1_ENDPOINTS) as pmod:
            for _ in range(5):
                x, y, z = acc.read()
                print(f'ACC: x={x:.03f}, y={y:.03f}, z={z:.03f}')

                x, y, z = mag.read()
                print(f'MAG: x={x:.03f}, y={y:.03f}, z={z:.03f}')    

            print('forward')
            pmod.start(10, PMODDirection.Forward)

            for _ in range(5):
                x, y, z = acc.read()
                print(f'ACC: x={x:.03f}, y={y:.03f}, z={z:.03f}')
                acc_history.append((x, y, z))

                x, y, z = mag.read()
                print(f'MAG: x={x:.03f}, y={y:.03f}, z={z:.03f}')    
                mag_history.append((x, y, z))
        
        acc_history = np.array(acc_history)
        mag_history = np.array(mag_history)

[2022-10-25 16:51:56,108] [INFO] upload firmware 'C:\ytliu2\ECE437\Lab7\Lab7.runs\impl_1\lab7_top.bit'
serial=1911000P3V
CRA_REG_M=10011100
ACC: x=1.012, y=-0.016, z=-0.036
MAG: x=-0.463, y=-0.858, z=0.892
ACC: x=1.000, y=-0.016, z=-0.040
MAG: x=-0.458, y=-0.861, z=0.893
ACC: x=1.000, y=-0.004, z=-0.036
MAG: x=-0.467, y=-0.861, z=0.895
ACC: x=1.012, y=-0.012, z=-0.040
MAG: x=-0.461, y=-0.860, z=0.895
ACC: x=1.004, y=-0.024, z=-0.020
MAG: x=-0.461, y=-0.861, z=0.893
forward
[2022-10-25 16:51:57,275] [INFO] control_reg=00000000000000000000000000001010
ACC: x=1.004, y=-0.008, z=-0.040
MAG: x=-0.461, y=-0.858, z=0.895
ACC: x=1.000, y=-0.012, z=-0.036
MAG: x=-0.462, y=-0.860, z=0.892
ACC: x=0.996, y=-0.004, z=-0.032
MAG: x=-0.458, y=-0.857, z=0.896
ACC: x=0.992, y=-0.004, z=-0.036
MAG: x=-0.461, y=-0.857, z=0.891
ACC: x=0.996, y=-0.012, z=-0.032
MAG: x=-0.457, y=-0.857, z=0.896


In [4]:
print('acc_mean')
print(abs(acc_history).mean(axis=0))
print('acc_max')
print(abs(acc_history).max(axis=0))


acc_mean
[0.9976 0.008  0.0352]
acc_max
[1.004 0.012 0.04 ]
